In [1]:
import numpy as np
from scipy.stats import norm

In [55]:
class LFSR:
    def __init__(self,n_reg, params, states):
        self.n_reg  = n_reg
        self.params = np.array(params, dtype=np.bool)
        self.states = np.array(states, dtype=np.bool)
    
    def set_params(self, params):
        self.params = params
        
    def set_current_state(self, state):
        self.states = state
    
    def iterate(self):
        current_output = self.states[0]
        next_last_state = np.logical_xor.reduce(self.params*self.states)
        self.states = np.roll(self.states, -1)
        self.states[-1] = next_last_state
        return current_output

class Geffe:
    def __init__(self, lfsr1, lfsr2, lfsr3):
        self.lfsr1 = lfsr1
        self.lfsr2 = lfsr2
        self.lfsr3 = lfsr3
    
    def one_iteration(self):
        x = self.lfsr1.iterate()
        y = self.lfsr2.iterate()
        s = self.lfsr3.iterate()
        return (s*x) ^ ((s^1)* y)
    
    def iterate(self, n):
        outputs = []
        for i in range(n):
            outputs.append(self.one_iteration())
        return np.array(outputs, dtype = np.bool)
    
def find_beta(n):
    return 1.0 / 2.0**n

def find_N_C(p1,p2,alpha,beta):
    a = p1 - p2
    b = np.sqrt(p1*(1-p1))*norm.ppf(1-alpha)+np.sqrt(p2*(1-p2))*norm.ppf(1-beta)
    c = 0
    d = np.sqrt(b**2-4*a*c)
    N1 = np.power((-b + d)/(2*a), 2)
    N2 = np.power((-b - d)/(2*a), 2)
    N = np.maximum(N1,N2)
    C = N*p1 + np.sqrt(N*p1*(1-p1))*norm.ppf(1-alpha)
    return N,C

def check_N_C(N,C, p1, p2, alpha, beta):
    return C - N*p1  - np.sqrt(N*p1*(1-p1))*norm.ppf(1-alpha), C  - N*p2 + np.sqrt(N*p2*(1-p2))*norm.ppf(1-beta)

In [44]:
n_lfsr1 = 12
n_lfsr2 = 10
n_lfsr3 = 11

lfsr_p1 = np.array([1,0,0,0,0,0,0,0,0,1,0,1], dtype=np.bool)
lfsr_p2 = np.array([1,0,0,0,0,1,1,0,1,1], dtype=np.bool)
lfsr_p3 = np.array([1,0,0,0,0,0,0,1,0,0,1], dtype=np.bool)
alpha   = 0.01
p1      = 1.0/4
p2      = 1.0/2

#var 3
sequence_str = """01101000101000101110010001100100001000100000010001111100100011001001011011010001011011101010001100010100000010001011111001101010101111011000100011110100110110011011100001010101101010011110000011000100110111011010111101100101010011110001010011001100111110010000000110100011101110001110000010101101000100010100000100011100111011111110010100001000101011111101011001001001010110111111110000010111100111100100110110110001011011111101101010001010100011001100110010011100010010000110110110101101110100010010"""
sequence = np.array([int(v) for v in sequence_str],dtype=np.bool)

LFSR1 = LFSR(len(lfsr_p1), lfsr_p1, [])
LFSR2 = LFSR(len(lfsr_p2), lfsr_p2, [])
LFSR3 = LFSR(len(lfsr_p3), lfsr_p3, [])

In [45]:
lfsr_params1 = np.array([1,1,1,1,1,1], dtype=np.bool)
lfsr_params2 = np.array([1,1,1,1], dtype=np.bool)
lfsr_params3 = np.array([1,1,1,1,1,1,1,1], dtype=np.bool)

lfsr_states1 = np.array([1,1,1,1,1,1], dtype=np.bool)
lfsr_states2 = np.array([1,1,1,1], dtype=np.bool)
lfsr_states3 = np.array([1,1,1,1,1,1,1,1], dtype=np.bool)

In [46]:
g = Geffe(LFSR1, LFSR2, LFSR3)

In [47]:
g.iterate(10)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [48]:
norm.ppf(0.975)

1.959963984540054

In [49]:
beta = find_beta(10)

In [56]:
N,C = find_N_C(1.0/4, 1.0/2, 0.01, beta)
print(N, C)

104.527944528 36.4309017383
